In [ ]:
import pandas as pd
import os
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
import rasterio
import numpy as np
import datetime as dt
from rasterstats import zonal_stats
from matplotlib import pyplot

oldpwd = os.getcwd()

In [ ]:
mandal_boundaries = gpd.read_file('../../../../../../src/data_preprocessing/tsdm/Mandal_Boundary.shp')
mandal_boundaries = mandal_boundaries.reset_index()

shape_telangana = gpd.read_file('../../../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')
shape_telangana = shape_telangana.explode()


In [ ]:
def crop_counts(boundaries, year):
    tiff = '01-01-'+str(year)+'.tif'
    lulc = rasterio.open(tiff, mode = 'r')
    lulc_array = lulc.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster
    # affine: 1: corresponds to the width of each pixel, 2: row rotation, 3: x-coordinate of the upper left pixel, 4: column rotation, 5: height of each pixel, 6: y-coordinate of the upper left pixel
    affine = lulc.transform

    cmap = {1: 'Water', 2: 'Trees', 4: 'Flooded Vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare Ground', 9: 'Snow/Ice', 10: 'Clouds', 11: 'Rangeland'}
    boundaries['Crops ' + str(year)] = 0
    boundaries['Flooded Vegetation ' + str(year)] = 0

    test = zonal_stats(boundaries.geometry, lulc_array, affine = affine, geojson_out = True, nodata = lulc.nodata, categorical=True, category_map = cmap)

    for i in range(0, len(test)):
        try:
            boundaries['Crops ' + str(year)].iloc[i] = test[i]['properties']['Crops']  * 0.0001
        except:
            boundaries['Crops ' + str(year)].iloc[i] = 0
        try:
            boundaries['Flooded Vegetation ' + str(year)].iloc[i] = test[i]['properties']['Flooded Vegetation']  * 0.0001
        except:
            boundaries['Flooded Vegetation ' + str(year)].iloc[i] = 0
        
    return (boundaries)

In [ ]:
for a in range(2017, 2020):
    mandal_boundaries = crop_counts(mandal_boundaries, a)
    print('Year ' + str(a) + ' is completed.')

In [ ]:
crops_over_time_mandal = mandal_boundaries[['geometry', 'Crops 2017', 'Crops 2018', 'Crops 2019', 'Flooded Vegetation 2017', 'Flooded Vegetation 2018', 'Flooded Vegetation 2019' ]]


In [ ]:
crops_over_time_mandal['2017'] = crops_over_time_mandal['Crops 2017'] + crops_over_time_mandal['Flooded Vegetation 2017']
crops_over_time_mandal['2018'] = crops_over_time_mandal['Crops 2018'] + crops_over_time_mandal['Flooded Vegetation 2018']
crops_over_time_mandal['2019'] = crops_over_time_mandal['Crops 2019'] + crops_over_time_mandal['Flooded Vegetation 2019']

In [ ]:
#crops_over_time_mandal = crops_over_time_mandal.reset_index()
fig, (axs) = pyplot.subplots(ncols=3, figsize=(12,4))
axs=axs.ravel()

axs[0].axis('off')
axs[0].set_title('2017')
crops_over_time_mandal.plot(ax = axs[0], column = '2017')

axs[1].axis('off')
axs[1].set_title('2018')
crops_over_time_mandal.plot(ax = axs[1], column = '2018')

axs[2].axis('off')
axs[2].set_title('2019')
crops_over_time_mandal.plot(ax = axs[2], column = '2019')

patch_col = axs[0].collections[0]
cb = fig.colorbar(patch_col, ax=axs, shrink = 0.75)
fig.suptitle('Crop Area km2')
plt.show()

In [ ]:
plt.plot(['2017', '2018', '2019'], [crops_over_time_mandal['2017'].sum(), crops_over_time_mandal['2018'].sum() , crops_over_time_mandal['2019'].sum()])
plt.title('Area Crops Over Time')
plt.ylim([0, 80000])
plt.xlabel('Year')
plt.ylabel('Area km2')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,18))

for i in range(0, len(mandal_boundaries)):
    x_axis = ['01-01-2017', '01-01-2018', '01-01-2019', '01-01-2020', '01-01-2021']
    y_axis = crops_over_time[i].to_list()
    plt.plot(x_axis, y_axis)

plt.title('Amount of Crop Area per Mandal')
plt.xlabel('Date')
plt.ylabel('Crop Area (Amount of pixels)')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller     
from statsmodels.tsa.stattools import kpss

adf_test = []
kpss_test = []
for i in range(0, len(mandal_boundaries)):
    X = crops_over_time[i]
    result = adfuller(X)
    adf_test.append(result[1])

    try:
        kpsstest = kpss(X, regression='c', nlags="auto")
        kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','#Lags Used'])
        kpss_test.append(kpss_output[1])
    except:
        kpss_test.append(0.1)


In [ ]:
fig = plt.figure(figsize = (20,18))

for i in range(0, len(mandal_boundaries)):
    kpss_test[i]
    if (kpss_test[i]<0.05) & (adf_test[i]>0.05):
        x_axis = ['01-01-2017', '01-01-2018', '01-01-2019', '01-01-2020', '01-01-2021']
        y_axis = crops_over_time[i].to_list()
        plt.plot(x_axis, y_axis)

plt.title('Amount of Crop Area per Mandal')
plt.xlabel('Date')
plt.ylabel('Crop Area (Amount of pixels)')
plt.show()

In [ ]:
crops_over_time.mean(axis = 1).plot()
plt.ylim(1000000, 1300000)
plt.title('Average Amount of Crops per Mandal')
plt.ylabel('Crops (Amount of Pixels)')
plt.show()
